# IDK MRC
run_qa.py

In [ ]:
!pip install -qqq transformers simpletransformers nltk stanza

     |████████████████████████████████| 5.8 MB 25.8 MB/s 
     |████████████████████████████████| 250 kB 90.3 MB/s 
     |████████████████████████████████| 691 kB 73.3 MB/s 
     |████████████████████████████████| 182 kB 76.8 MB/s 
     |████████████████████████████████| 7.6 MB 69.9 MB/s 
     |████████████████████████████████| 43 kB 2.0 MB/s 
     |████████████████████████████████| 452 kB 74.4 MB/s 
     |████████████████████████████████| 9.2 MB 71.2 MB/s 
     |████████████████████████████████| 1.3 MB 66.1 MB/s 
     |████████████████████████████████| 1.9 MB 33.8 MB/s 
     |████████████████████████████████| 182 kB 79.5 MB/s 
     |████████████████████████████████| 174 kB 45.8 MB/s 
     |████████████████████████████████| 62 kB 1.5 MB/s 
     |████████████████████████████████| 173 kB 82.0 MB/s 
     |████████████████████████████████| 168 kB 82.4 MB/s 
     |████████████████████████████████| 168 kB 86.8 MB/s 
     |████████████████████████████████| 166 kB 83.7 MB/s 
     |████████████

In [ ]:
!cp -r /content/drive/MyDrive/IDK-MRC/dataset .

In [ ]:
import argparse
import json
import os
import torch
import wandb
from transformers.utils import logging
from simpletransformers.question_answering import QuestionAnsweringModel
from squad_2_0_eval import evaluate, compute_f1

In [ ]:
wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
torch.multiprocessing.set_sharing_strategy('file_system')
logger = logging.get_logger(__name__)

In [ ]:
def get_model(args, train_args, pretrained=None):
    if 'indobert' in args.pretrain_name:
        train_args['config'] = {
            'num_labels': 2,
            'id2label': {
                "0": "LABEL_0",
                "1": "LABEL_1"
            },
            'label2id': {
                "LABEL_0": 0,
                "LABEL_1": 1
            }
        }

    if pretrained is None:
        pretrained = args.pretrain_name

    # use first available gpu
    cuda_device = -1 if 'n_gpu' in train_args and train_args['n_gpu'] > 1 else args.device_num

    return QuestionAnsweringModel(args.model_name, pretrained, args=train_args, cuda_device=cuda_device, use_cuda=torch.cuda.is_available())

In [ ]:
def eval_qa(model, val_path, eval_output_dir, filename='eval_result'):
    if not os.path.exists(eval_output_dir):
        os.makedirs(eval_output_dir)

    result, text = model.eval_model(val_path, output_dir=eval_output_dir)
    exact_match, f1, result_squad = evaluate(val_path, eval_output_dir + 'predictions_test.json')
    result['exact'] = exact_match
    result['f1'] = f1

    # Save result
    with open(eval_output_dir + filename + '.json', 'w') as f:
        json.dump(result, f)

    with open(eval_output_dir + filename + '_text.json', 'w') as f:
        json.dump(text, f)

    with open(eval_output_dir + filename + '_squad_2.0.json', 'w') as f:
        json.dump(result_squad, f)

    return result, text, result_squad

In [ ]:
def hasans_f1_metric(true_answers, predicted_answers):
    f1_scores = []
    for true_ans, pred_ans in zip(true_answers, predicted_answers):
        if len(true_ans) > 0:
            f1 = compute_f1(true_ans, pred_ans)
            f1_scores.append(f1)
    return 100.0 * sum(f1_scores) / len(f1_scores)

In [ ]:
def avg_f1_metric(true_answers, predicted_answers):
    ans_f1_scores, unans_f1_scores = [], []
    for true_ans, pred_ans in zip(true_answers, predicted_answers):
        f1 = compute_f1(true_ans, pred_ans)
        if len(true_ans) > 0:
            ans_f1_scores.append(f1)
        else:
            unans_f1_scores.append(f1)
    ans_f1 = 100.0 * sum(ans_f1_scores) / len(ans_f1_scores)
    unans_f1 = 100.0 * sum(unans_f1_scores) / len(unans_f1_scores)
    return (ans_f1 + unans_f1) / 2

In [ ]:
def train_qa(args, train_args, train_path, eval_path):
    
    print("WandB initialize...")
    wandb.init(project=args.wb_name)
    wandb.run.name = args.out_name
    wandb.run.save()
    
    print("Get model...")
    model = get_model(args, train_args)
    
    print("train_model()")
    model.train_model(train_path, eval_data=eval_path, ans_f1=hasans_f1_metric)

In [ ]:
args = argparse.Namespace(
    train_path="dataset/idk_mrc/train.json",
    val_path="dataset/idk_mrc/valid.json",
    test_path="dataset/idk_mrc/test.json",
    model_name="bert",
    pretrain_name="indobenchmark/indobert-base-p2",
    epoch=10,
    device_num=0,
    n_gpu=1,
    wb_name="idk-mrc",
    wb_name_test="idk-mrc-test",
    out_path="outputs",
    out_name="indobert_squad",
    lang="id",
    seed=42,
    uncased=True,
    only_test=False
)

In [ ]:
print("Running model for seed:", str(args.seed))
print("Input file:", args.train_path)

Running model for seed: 42
Input file: dataset/idk_mrc/train.json


In [ ]:
output_dir = args.out_path + '/' + args.wb_name + '/' + args.model_name + '/' + args.out_name + '/' + str(args.seed) + '/'
train_args = {
    'learning_rate': 2e-5,
    'num_train_epochs': args.epoch,
    'max_seq_length': 512,
    'max_query_length': 128,
    'doc_stride': 128,
    'overwrite_output_dir': True,
    'reprocess_input_data': True,
    'train_batch_size': 16,
    'fp16': True,
    'output_dir': output_dir,
    'wandb_project': args.wb_name,
    'n_best_size': 20,
    'manual_seed': args.seed,
    'encoding': 'utf-8',
    'save_eval_checkpoints': False,
    'save_model_every_epoch': False,
    'save_steps': -1,
    'do_lower_case': args.uncased,
    'adam_epsilon': 1e-8,
    'n_gpu': args.n_gpu,
    'best_model_dir': output_dir + 'best_model/',
    'evaluate_during_training': True,
    'evaluate_during_training_steps': 2000,
    'early_stopping_metric': 'ans_f1',
    'early_stopping_metric_minimize': False,
    'no_cache': True
}

# Train the model

In [ ]:
if not args.only_test:
    print("Start training...")
    train_qa(args, train_args, args.train_path, args.val_path)
else:
    wandb.init(project=args.wb_name_test)
    wandb.run.name = args.out_name
    wandb.run.save()

wandb: Currently logged in as: andreaschandra. Use `wandb login --relogin` to force relogin


Start training...
WandB initialize...


wandb: WARNING Calling wandb.run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


Get model...


Downloading:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

...


convert squad examples to features: 100%|██████████| 764/764 [00:04<00:00, 161.57it/s]

add example index and unique id: 100%|██████████| 764/764 [00:00<00:00, 407846.28it/s]


Running Evaluation:   0%|          | 0/96 [00:00<?, ?it/s]

Running Epoch 9 of 10:   0%|          | 0/592 [00:00<?, ?it/s]


convert squad examples to features: 100%|██████████| 764/764 [00:04<00:00, 164.58it/s]

add example index and unique id: 100%|██████████| 764/764 [00:00<00:00, 318533.62it/s]


Running Evaluation:   0%|          | 0/96 [00:00<?, ?it/s]

# Evaluate the model

In [ ]:
model = get_model(args, train_args, pretrained=output_dir + 'best_model/')

In [ ]:
_, _, result_squad = eval_qa(model, args.test_path, output_dir + 'test/')
wandb.log({
    'HasAns_exact': result_squad['HasAns_exact'],
    'HasAns_f1': result_squad['HasAns_f1'],
    'NoAns_exact': result_squad['NoAns_exact'] if 'NoAns_exact' in result_squad else 0,
    'NoAns_f1': result_squad['NoAns_f1'] if 'NoAns_f1' in result_squad else 0,
    'exact': result_squad['exact'],
    'f1': result_squad['f1']
})

add example index and unique id: 100%|██████████| 844/844 [00:00<00:00, 325786.17it/s]


Running Evaluation:   0%|          | 0/107 [00:00<?, ?it/s]

In [ ]:
result_squad

OrderedDict([('exact', 71.56398104265404),
             ('f1', 77.62447627362148),
             ('total', 844),
             ('HasAns_exact', 57.81990521327014),
             ('HasAns_f1', 69.94089567520503),
             ('HasAns_total', 422),
             ('NoAns_exact', 85.30805687203791),
             ('NoAns_f1', 85.30805687203791),
             ('NoAns_total', 422)])